In [1]:
import pandas as pd
import numpy as np

# Models
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn.metrics as metrics
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
from varname import nameof
from sklearn.metrics import matthews_corrcoef

np.set_printoptions(precision=3)
pd.set_option('precision', 3)

sns.set()

### Клиенты телекоммуникационной компании 

У вас есть набор данных от некой телекоммуникационной компании. Данные содержат информацию о пользователях компании, их демографических характеристиках, услугах, которыми они пользуются, продолжительности пользования услугами компании, способе оплаты и размере оплаты.

Задача состоит в том, чтобы проанализировать данные и спрогнозировать отток пользователей (выявить людей, которые будут и не будут продлевать свой контракт с компанией).


Gender - пол (male / female) <br>
Senior - пенсионер или нет (1, 0)<br>
Married - состоит ли в браке (Yes, No) <br>
Tenure - сколько месяцев человек является клиентом компании<br>
Phone - подключен ли телефон (Yes, No)<br>
MultiplePhone - подключено ли несколько телефонных каналов (Yes, No, No phone service)<br>
Internet - предоставляются ли услуги интернет соединения (DSL, Fiber optic, No)<br>
Security - подключен ли сервис интернет-безопасности (Yes, No, No internet service)<br>
Backup - активирована ли служба онлайн-резервного копирования (Yes, No, No internet service)<br>
Insurance - есть ли у клиента страховка оборудования (Yes, No, No internet service)<br>
Support - подключена ли служба технической поддержки (Yes, No, No internet service)<br>
TV - подключена ли служба потокового телевидения (Yes, No, No internet service)<br>
Movies - активирован ли сервис потокового кино (Yes, No, No internet service)<br>
Contract - тип клиентского контракта (Month-to-month, One year, Two year)<br>
EBilling - использует ли клиент безбумажный биллинг (Yes, No)<br>
PaymentMethod - способ оплаты (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))<br>
MonthlyCharges - текущий ежемесячный платеж<br>
TotalCharges - общая сумма, которую клиент заплатил за услуги за все время<br>
Churn - покинул ли клиент компанию (Yes or No)<br>

In [2]:
test = pd.read_csv("test.csv", index_col=0)

In [3]:
data = pd.read_csv('train.csv', index_col=0)

In [4]:
# Поле для экспериментов

In [5]:
data.TotalCharges = data.TotalCharges.replace(' ', 0).astype(float)

In [6]:
test.TotalCharges = test.TotalCharges.replace(' ', 0).astype(float)

In [7]:
data = data.replace({'Gender': {'Female': 1, 'Male': 0}, 
                     #'Married': {'Yes': 1, 'No': 0}, 
                     'Phone': {'Yes': 1, 'No': 0},
                     #'EBilling': {'Yes': 1, 'No': 0}, 
                     #'Dependents': {'Yes': 1, 'No': 0}, 
                     'MultiplePhones': {'Yes': 1, 'No': 0, 'No phone service': 0}, 
                     'Security': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Backup': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Insurance': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Support': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'TV': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Movies': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Contract': {'Month-to-month': 0, 'One year': 1, 'Two year': 2}, 
                     'Internet': {'Fiber optic': 1, 'DSL': 2, 'No': 0}, 
                    })

In [8]:
test = test.replace({'Gender': {'Female': 1, 'Male': 0}, 
                     #'Married': {'Yes': 1, 'No': 0}, 
                     'Phone': {'Yes': 1, 'No': 0},
                     #'EBilling': {'Yes': 1, 'No': 0}, 
                     #'Dependents': {'Yes': 1, 'No': 0}, 
                     'MultiplePhones': {'Yes': 1, 'No': 0, 'No phone service': 0}, 
                     'Security': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Backup': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Insurance': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Support': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'TV': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Movies': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Contract': {'Month-to-month': 0, 'One year': 1, 'Two year': 2}, 
                     'Internet': {'Fiber optic': 1, 'DSL': 2, 'No': 0}, 
                    })

In [9]:
# Выбираем только числовые столбцы

In [10]:
train_numeric_data = data.select_dtypes(include=np.number)
test_numeric_data = test.select_dtypes(include=np.number)

In [11]:
# Обучаем модели

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(train_numeric_data, 
                                                    data.Churn,
                                                    test_size = 0.20,
                                                    random_state = 1234)


In [356]:

std_scaler = StandardScaler()

X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.transform(X_test)

X_train

array([[-0.966, -0.447, -0.978, ...,  0.831,  0.351,  0.874],
       [-0.966, -0.447,  1.022, ..., -1.203,  0.402,  1.407],
       [ 1.035, -0.447,  1.022, ..., -1.203, -1.499, -0.833],
       ...,
       [-0.966, -0.447,  1.022, ..., -1.203, -1.305, -0.677],
       [ 1.035, -0.447,  1.022, ..., -1.203, -1.522, -0.807],
       [-0.966, -0.447,  1.022, ...,  0.831,  0.164, -0.719]])

In [13]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(X_train, Y_train)

dt_train_predictions = dt.predict(X_test)

matthews_corrcoef(Y_test, dt_train_predictions)

0.31854392732905773

In [14]:
smart_dt = DecisionTreeClassifier(min_samples_leaf = 7, max_depth = 17).fit(X_train, Y_train)
print (f'Tree depth: {smart_dt.get_depth()}')

print('Test evaluation')
sdt_test_predictions = smart_dt.predict (X_test)
matthews_corrcoef(Y_test, sdt_test_predictions)

Tree depth: 16
Test evaluation


0.33977938486425113

In [15]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

tree_params = {'min_samples_leaf': [5, 7, 9, 11],
               'max_depth': [12, 15, 17, 19],
               'criterion': ['gini', 'entropy']              
              }

grid = GridSearchCV(DecisionTreeClassifier(),
                    tree_params,
                    refit=True,
                    scoring='f1_macro',
                    n_jobs = -1,
                    cv=StratifiedKFold(n_splits = 5),
                    verbose=2
                   )

grid_best_model = grid.fit(X_train, Y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


In [16]:
print('Test evaluation')
gbm_test_predictions = grid_best_model.predict (X_test)
matthews_corrcoef(Y_test, gbm_test_predictions)

Test evaluation


0.39140775132924416

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

lr = LogisticRegression()

cv_lr_predictions = cross_val_predict(lr, X_train, Y_train, cv = 5, n_jobs = -1)


lr = LogisticRegression().fit(X_train, Y_train)
print('Test evaluation')
lr_test_predictions = lr.predict (X_test)
matthews_corrcoef(Y_test, lr_test_predictions)

Test evaluation


0.46563816352707077

In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, Y_train)

y_pred = knn.predict(X_test)
matthews_corrcoef(Y_test, y_pred)

0.3410618845604188

In [19]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train,Y_train)

Y_pred_nb = nb.predict(X_test)

In [20]:
matthews_corrcoef(Y_test, Y_pred_nb)

0.4472228306879629

In [21]:
# Получаем предсказания для тестовых данных

In [22]:
predictions_test = lr.predict (test_numeric_data)

In [23]:
# Сохраняем данные

In [24]:
predictions_df = pd.DataFrame(data=predictions_test, columns=["Churn"])

In [25]:
predictions_df

,Churn
0,False
1,True
2,False
3,False
4,True
...,...
2390,False
2391,False
2392,True
2393,True


In [26]:
predictions_df.Churn.value_counts()

False    1894
True      501
Name: Churn, dtype: int64

In [27]:
predictions_df.to_csv(index=True, index_label='Index', path_or_buf='sample_submission.csv')